In [132]:
import os, email
from imapclient.response_types import Address
import email
from email.header import decode_header, make_header
from datetime import date
from imapclient import IMAPClient
from email.header import decode_header
from email import message_from_bytes
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

USERNAME = os.getenv("user")
PASSWORD = os.getenv("pass")

In [133]:
server = IMAPClient('imap.gmail.com', use_uid=True)

server.login(USERNAME, PASSWORD)
select_info = server.select_folder('INBOX')
print('%d messages in INBOX' % select_info[b'EXISTS'])

6852 messages in INBOX


In [134]:
l=server.search(criteria=[u'SINCE', date(2025, 8, 6)])     #year,month,day
#l=server.search(criteria=b'SINCE 06-Aug-2025')

In [135]:
#https://imapclient.readthedocs.io/en/3.0.0/api.html#imapclient.response_types.Envelope    
# for msgid, data in server.fetch(l[-3], ['BODYDATA']).items():
#     envelope = data[b'BODYDATA']
#     print(data)
    #name=envelope.sender[0].name.decode()
    #cur_email = f"{envelope.sender[0].mailbox.decode()}@{envelope.sender[0].host.decode()}"#envelope.sender[0].mailbox.decode()+'@'+
    #print('ID %d: "%s" received %s' % (msgid, envelope.subject.decode(), envelope.date))

In [136]:
def parse_email_address(addr: Address):
    if not addr:
        return None, None
    name = addr.name.decode() if addr.name else None
    email_addr = f"{addr.mailbox.decode()}@{addr.host.decode()}" if addr.mailbox and addr.host else None
    return name, email_addr

In [137]:
for msgid, data in server.fetch(l[-1], ['ENVELOPE', 'BODY[]']).items():
    # Extract envelope info
    sender = data[b'ENVELOPE'].sender[0]  # type: Address
    name, email_addr = parse_email_address(sender)
    
    # Get raw email and parse body
    msg = email.message_from_bytes(data[b'BODY[]'])
    # Extract body
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            if "attachment" not in content_disposition and content_type == "text/plain":
                body = part.get_payload(decode=True).decode()
                break
    else:
        # Non-multipart email
        charset =msg.get_content_charset()
        body = msg.get_payload(decode=True).decode()

    # Print the result
    print("Sender Name:", name)
    print("Sender Email:", email_addr)
    print("Body:\n", body)

Sender Name: Microsoft Azure Team
Sender Email: azure@email.microsoft.com
Body:
 Explore best practices, use cases and solutions to help you get started.

Having trouble viewing this email? | View as a web page <[[https://emails.microsoft.com/v/MTU3LUdRRS0zODIAAAGcHuf6ez6pai3geq3pKTl6DFth--yD4YVudAbwjBeJlPqPn7ECWJFohg39ukb8zHyHxbci13o=]]>

<[[https://azure.microsoft.com]]>
Lead your team to AI innovation with resources from Azure  


 Discover the latest innovations in cloud and AI, advance your generative AI efforts and confidently lead your organisation to long-term success – by accessing the latest resources from Azure.
Customizing generative AI for unique value

Learn how global leaders are using AI to drive value in this report from Microsoft and MIT Technology Review Insights – and discover ways to customise your own solutions.

Read the report <https://emails.microsoft.com/dc/pTEFlGZ9Q3ITrVt7_I2wJfSaXP4fVmu5GQerBk9DuUsq46jcuZX_ffRpaPr5IjSLrH4T195L07qohAiEBW0ylA==/MTU3LUdRRS0zODI

In [138]:
import sys
size_in_bytes=sys.getsizeof(body)

size_in_mb=size_in_bytes / (1024 * 1024)
print(f"Size in MB: {size_in_mb}")

Size in MB: 0.00724029541015625
